# Create Datasets for the Content-based Filter

This notebook builds the data you will use for creating our content based model. You'll collect the data via a collection of SQL queries from the publicly available Kurier.at dataset in BigQuery.
Kurier.at is an Austrian newsite. The goal of these labs is to recommend an article for a visitor to the site. In this notebook, you collect the data for training, in the subsequent notebook you train the recommender model. 

This notebook illustrates:
* How to pull data from BigQuery table and write to local files.
* How to make reproducible train and test splits.

In [19]:
import os
import tensorflow as tf
import numpy as np
from google.cloud import bigquery 

PROJECT = 'qwiklabs-gcp-03-9e74f26a3ed5' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-03-9e74f26a3ed5' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [20]:
### need to set google-api-core==1.33.2
!pip install google-api-core==1.33.2

In [21]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


You will use this helper function to write lists containing article ids, categories, and authors for each article in our database to local file. 

In [22]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(line)

### Pull data from BigQuery

The cell below creates a local text file containing all the article ids (i.e. 'content ids') in the dataset. 

Have a look at the original dataset in [BigQuery](https://console.cloud.google.com/bigquery?p=cloud-training-demos&d=GA360_test&t=ga_sessions_sample). Then read through the query below and make sure you understand what it is doing. 

In [23]:
sql="""
#standardSQL

SELECT  
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  content_id
  
"""

content_ids_list = bigquery.Client().query(sql).to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")
print("Some sample content IDs {}".format(content_ids_list[:3]))
print("The total number of articles is {}".format(len(content_ids_list)))

Some sample content IDs ['298846345', '297944011', '115827617']
The total number of articles is 15634


There should be 15,634 articles in the database.  
Next, you'll create a local file which contains a list of article categories and a list of article authors.

Note the change in the index when pulling the article category or author information. Also, you are using the first author of the article to create our author list.  
Refer back to the original dataset, use the `hits.customDimensions.index` field to verify the correct index.	 

In [24]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  category
"""
categories_list = bigquery.Client().query(sql).to_dataframe()['category'].tolist()
write_list_to_disk(categories_list, "categories.txt")
print(categories_list)

['Stars & Kultur', 'Lifestyle', 'News']


The categories are 'News', 'Stars & Kultur', and 'Lifestyle'.  
When creating the author list, you'll only use the first author information for each article.  

In [25]:
sql="""
#standardSQL
SELECT
  REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  first_author
"""
authors_list = bigquery.Client().query(sql).to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")
print("Some sample authors {}".format(authors_list[:10]))
print("The total number of authors is {}".format(len(authors_list)))

Some sample authors ['Elisabeth Spitzer', 'Raffaela Lindorfer', 'Peter Karlik', 'Georg Leyrer', 'Mathias Kainz', 'Stefan Berndl', 'Karl Oberascher', 'Klaus Knittelfelder', 'Moritz Gottsauner-Wolf', 'Daniela Wahl']
The total number of authors is 385


There should be 385 authors in the database. 

### Create train and test sets

In this section, you will create the train/test split of our data for training our model. You use the concatenated values for visitor id and content id to create a farm fingerprint, taking approximately 90% of the data for the training set and 10% for the test set.

In [26]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) < 9
"""
training_set_df = bigquery.Client().query(sql).to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1049534290319968384,299946732,News,"""Explosiver"" Mieter soll aus Groll seine Wohnu...",Ricardo Peyerl,574,299946732
1,1049534290319968384,299946732,News,"""Explosiver"" Mieter soll aus Groll seine Wohnu...",Ricardo Peyerl,574,299969709
2,1049534290319968384,268707300,Lifestyle,Manspreading: Madrid geht gegen breitbeinige S...,Marlene Patsalidis,569,268707300
3,1049534290319968384,268707300,Lifestyle,Manspreading: Madrid geht gegen breitbeinige S...,Marlene Patsalidis,569,299925700
4,1060199256712239175,299804373,Lifestyle,Bloggerin wegen Anstiftung zur Magersucht ange...,Elisabeth Mittendorfer,574,299772450


In [27]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) >= 9
"""
test_set_df = bigquery.Client().query(sql).to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1019158988689919195,299833903,News,Ungarns Regierung kämpft mit Buch gegen Soros,Peter Temel,574,299816215
1,118772017798034869,299838010,News,Doch plus 233 Prozent für Großteil der Beamten,Wolfgang Atzenhofer,574,299451483
2,1228262559676487843,299818044,News,Lehrer am Gymnasium Schwechat suspendiert,None,574,272595271
3,1464829057720768794,254619647,None,ANB,None,566,299826775
4,1555557450090689676,299818044,News,Lehrer am Gymnasium Schwechat suspendiert,None,574,299853016


Let's have a look at the two csv files you just created containing the training and test set. You'll also do a line count of both files to confirm that you have achieved an approximate 90/10 train/test split.  
In the next notebook, **Content Based Filtering** you will build a model to recommend an article given information about the current article being read, such as the category, title, author, and publish date. 

In [28]:
%%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [29]:
!head *_set.csv

==> test_set.csv <==
1019158988689919195,299833903,News,Ungarns Regierung kämpft mit Buch gegen Soros,Peter Temel,574,299816215
118772017798034869,299838010,News,Doch plus 233 Prozent  für Großteil der Beamten,Wolfgang Atzenhofer,574,299451483
1228262559676487843,299818044,News,Lehrer am Gymnasium Schwechat suspendiert,,574,272595271
1464829057720768794,254619647,,ANB,,566,299826775
1555557450090689676,299818044,News,Lehrer am Gymnasium Schwechat suspendiert,,574,299853016
1555557450090689676,299435706,Lifestyle,Diese Bloggerin weiß wie man Fehlkäufe vermeidet,Gabriele Kuhn,574,299943437
1964842686378339773,299828023,News,Glyphosat geht in die Verlängerung,Andreas Anzenberger,574,299831571
2036370972892179839,299817990,News,Kolumne Anstoß: Das Allerheiligste,Philipp Albrechtsberger,574,299939900
210110321307411584,299809186,Lifestyle,Ugly Sweater-WM: Wer hat den hässlichsten Weihnachts-Look?,Maria Zelenko,574,299410466
2292459398710009319,299939897,News,Drohnen-Sicherheitspaket präsent